Import Libraries:

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import  layers, models
#from tensorflow.python.keras import layers, models
import os
from skimage import io, color, transform
import numpy as np

import os
from skimage import io, color, transform
import numpy as np


2024-01-17 09:16:26.106973: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-17 09:16:26.505454: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-17 09:16:26.505655: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-17 09:16:26.573836: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-17 09:16:26.723827: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-17 09:16:26.727001: I tensorflow/core/platform/cpu_feature_guard.cc:1

Define Constants:

In [4]:
# Specify the path to your dataset

dataset_path = "/home/zkerroumi42/ProjetIA-ZNZ/dataset"

class_labels = ["Benign_Keratosis_(BKL)", "Molluscum", "Seborrheic_Keratoses"]
image_height, image_width = 224, 224
num_classes = len(class_labels)

Data Preprocessing:

In [2]:
def load_images_and_labels(folder_path):
    images = []
    labels = []
    for class_label, folder_name in enumerate(class_labels):
        class_folder = os.path.join(folder_path, folder_name)
        for filename in os.listdir(class_folder):
            image_path = os.path.join(class_folder, filename)
            image = io.imread(image_path)
            image = color.rgb2gray(image)  # Convert to grayscale
            image = transform.resize(image, (image_height, image_width))  # Resize the image
            images.append(image.flatten())  # Flatten the image
            labels.append(class_label)
    return np.array(images), np.array(labels)

In [5]:

# Load and preprocess the images
X, y = load_images_and_labels(dataset_path)

# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the input data to fit the CNN model
import numpy as np
X_train = X_train.reshape(-1, image_height, image_width, 1)
X_test = X_test.reshape(-1, image_height, image_width, 1)

# Convert labels to one-hot encoding
from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train, num_classes)
y_test_one_hot = to_categorical(y_test, num_classes)


: 

Build the CNN Model:

In [5]:
from keras import  layers, models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))



Compile the Model

In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Train the Model:

In [7]:
history=model.fit(X_train, y_train_one_hot, epochs=10, validation_split=0.2)


Epoch 1/10


101/101 [==============================] - 155s 1s/step - loss: 0.8721 - accuracy: 0.5991 - val_loss: 0.7212 - val_accuracy: 0.6908
Epoch 2/10
101/101 [==============================] - 74s 734ms/step - loss: 0.5793 - accuracy: 0.7332 - val_loss: 0.5396 - val_accuracy: 0.7494
Epoch 3/10
101/101 [==============================] - 71s 702ms/step - loss: 0.4664 - accuracy: 0.7911 - val_loss: 0.4996 - val_accuracy: 0.7855
Epoch 4/10
101/101 [==============================] - 70s 695ms/step - loss: 0.4251 - accuracy: 0.8108 - val_loss: 0.4920 - val_accuracy: 0.7731
Epoch 5/10
101/101 [==============================] - 72s 715ms/step - loss: 0.3586 - accuracy: 0.8479 - val_loss: 0.4674 - val_accuracy: 0.7993
Epoch 6/10
101/101 [==============================] - 72s 714ms/step - loss: 0.3247 - accuracy: 0.8563 - val_loss: 0.5231 - val_accuracy: 0.7843
Epoch 7/10
101/101 [==============================] - 73s 729ms/step - loss: 0.2525 - accuracy: 0.8934 - val_loss: 0.6229 - val_ac

Saving the model and  Evaluate the model on the test set

In [8]:
# Save the model architecture to a JSON file
model_json = model.to_json()
with open("cnn_model_archi.json", "w") as json_file:
    json_file.write(model_json)

# Save the model weights to an HDF5 file
model.save_weights("cnn_model_weights.h5")

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test_one_hot)

print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

# Optionally, save the training history to a file
np.save("training_history.npy", history.history)

32/32 [==============================] - 4s 132ms/step - loss: 0.9015 - accuracy: 0.7537
Test Accuracy: 75.37%


 Evaluate the model on the test set